# Data clensing and SQL Data Calls

In [1]:
import pandas as pd
import numpy as np
import mysql.connector as sq

Dataframe Creation

In [2]:
df = pd.read_csv ('nj_teachers_salaries_pset4.csv')
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100005 entries, 0 to 100004
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   99998 non-null   float64
 1   last_name            100003 non-null  object 
 2   first_name           100003 non-null  object 
 3   county               100003 non-null  object 
 4   district             100003 non-null  object 
 5   school               100003 non-null  object 
 6   primary_job          100003 non-null  object 
 7   fte                  100003 non-null  object 
 8   salary               99983 non-null   object 
 9   certificate          100003 non-null  object 
 10  subcategory          100003 non-null  object 
 11  teaching_route       100003 non-null  object 
 12  highly_qualified     100003 non-null  object 
 13  experience_district  100003 non-null  object 
 14  experience_nj        100003 non-null  object 
 15  experience_total 

/var/folders/wh/8qjdz4h56bb3smy6brz1xr6c0000gn/T/ipykernel_16154/2225393921.py:3: DtypeWarning: Columns (7,8,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv ('nj_teachers_salaries_pset4.csv')


(100005, 16)

## Droping All NaN Values Row

In [3]:
df.dropna(how="all", inplace=True)
df.shape

(100003, 16)

## Finding and Culling Bad Numerical Entries

In [4]:
# Creating the functions from the test that can be used for any column starting with finding
# all the rows where an incorrect value is entered, inputing the name of the column as a string
# and which dataframe should be used, and then outputting a list of all wrong rows
def wrong_r(column, datfra):
    wrong_rows = []

    # Creating a for loop that will move through each entry in a column/field and check if it
    # contains anything other than 0-9, and if it does it will add that column to a list.
    for i in range(len(datfra)):

        # Since several completely blank rows have already been removed, the loop experiences
        # a range error when it runs into those rows, so first it will check if i matches one 
        # of the values from the dataframe and if it doesn't then that is a skipped row and
        # the loop will iterate immediately to prevent the range error and move onto the 
        # the next populated row, while keeping the index of the for-loop synced with the 
        # index of the dataframe.
        if i in datfra.index:
            # Creating a variable that will store the value from a certain cell so it can be 
            # checked if it contains anything other than digits
            value = datfra[column][i]

            # Checking digits. If non digit detected then adding the index number to the list.
            # Converting the value ot a string so that it can be correctly evaluated if the 
            # Column has already been converted to all ints or floats.
            if str(value).isdigit() == False:
                wrong_rows.append(i)

        # Iterating if i doesn't match index i.e. row missing
        else:
            continue

    return wrong_rows

# Printing the total quantity of incorrect entries and then printing the first 10 and
# and last 10 row numbers and values as a visual sample. If the data has already been fixed
# then it can also print the comparison. If no comparison then the same list should be listed
# twice when calling the function.
def print_wrong_sample (wrong_rows, new_wrong_rows, column, datfra, update):

    # Assigning the length of both the original list and potential updated lists to display
    wrong_rows_length = len(wrong_rows)
    new_wrong_rows_length = len(new_wrong_rows)

    # Creating a variable to create a print that will only print based on an earlier print function
    difference = 0

    print('Quantity of intially incorrect entries:', wrong_rows_length, '\n')

    if wrong_rows_length !=0:
    
        # Checking if the quantity was marked as updated and checking if their was a difference
        # in the ammount of incorrect entries. If so, it will display the new quantity.
        if (update == 'Y' or update == 'y') and wrong_rows != new_wrong_rows:
            print('Current Quantity of incorrect entries:', new_wrong_rows_length, '\n')
            difference = 1
            print('First and last 10 intially incorrect rows (with current values)')
    
        # If an update was requested but there wasn't a change in quantity then that will
        # be displayed to the user.
        elif update == 'Y' or update == 'y':
            print('Quanity of incorrect entries has not changed \n')
            print('First and last 10 incorrect rows (with current values)')
    
        else: 
            print('First and last 10 incorrect rows and values:')
            
        # Now displaying the first 10 and last 10 
        for item in wrong_rows[:10]+ wrong_rows[-10:]:
            print(item, datfra[column][item])
    
        print('\n')
    
        # Checking if previous check found a new quantity of incorrect entries, and then
        # Printing the first and last 10 entries of update
        if difference == 1:
            print('Current first and last 10 incorrect rows and values:')
            for item in new_wrong_rows[:10]+ new_wrong_rows[-10:]:
                print(item, datfra[column][item])
    
            print('\n')

# Creating a function to detect which of the incorrect entries in the column are floats,
# which will then be stored in a list which can be displayed. Additionally it will check
# if any of the float entries are ints that miscelleniously had '.0' tacked on causing them 
# to be read as floats and simply need to have the float part trimmed.
def float_r(column, datfra):
    float_rows = []
    non_zero_float = []
    zero_float = []

    # Creating loop to check each entry in the column to see if it contains a '.'
    for i in range(len(datfra)):

        # Following the same intial protocol of wrong_r and checking to make sure the indeces 
        # match
        if i in datfra.index:
            value = datfra[column][i]

            # Checking if the value is actually a float by detecting if the string version
            # of the value contains the character '.' is present but only once and if all
            # other symbols are digits.
            if '.' in str(value) and str(value).replace('.','',1).isdigit() == True:
                float_rows.append(i)

                # Separating out the part after the decimal place in order to detect if the 
                # the decimal portion of the value is only a single zero in the tenths place
                # and then likely good data just entered as the wrong data type
                decimal_part = str(value).split('.')[1]
                if '.0' not in str(value) or len(decimal_part) >=2:
                    # Saving the location of any entries that violate this check.
                    non_zero_float.append(i)

        else:
            continue

    # Creating an additional list of all the locations of just entries that seem to have correct
    # values with the '.0' added in case it is desired that these be specifically imputed
    zero_float = list(set(float_rows) - set(non_zero_float))

    return float_rows, non_zero_float, zero_float
    
def print_float_sample (float_rows, new_float_rows, column, datfra, update):
    float_rows_length = len(float_rows)
    
    # Assigning the length of both the original list and potential updated lists to display
    new_float_rows_length = len(new_float_rows)

    # Creating a variable to create a print that will only print based on an earlier print function
    difference = 0

    print('Quantity of intial float entries:', float_rows_length, '\n')

    if float_rows_length !=0:

        # Checking if the quantity was marked as updated and checking if their was a difference
        # in the ammount of float entries. If so, it will display the new quantity.
        if (update == 'Y' or update == 'y') and float_rows != new_float_rows:
            print('Current Quantity of float entries:', new_float_rows_length, '\n')
            difference = 1
            print('First and last 10 intial float rows (with current values)')
    
        # If an update was requested but there wasn't a change in quantity then that will
        # be displayed to the user.
        elif update == 'Y' or update == 'y':
            print('Quanity of float entries has not changed \n')
            print('First and last 10 float rows (with current values)')
    
        else: 
            print('First and last 10 float rows and values:')
            
        # Now displaying the first 10 and last 10 
        for item in float_rows[:10]+ float_rows[-10:]:
            print(item, datfra[column][item])
    
        print('\n')
    
        # Checking if previous check found a new quantity of float entries, and then
        # Printing the first and last 10 entries of update
        if difference == 1:
            print('Current first and last 10 float rows and values:')
            for item in new_float_rows[:10]+ new_float_rows[-10:]:
                print(item, datfra[column][item])
    
            print('\n')

def print_non_zero_float_sample (non_zero_float, new_non_zero_float, column, datfra, update):
    non_zero_float_length = len(non_zero_float)
    
    # Assigning the length of both the original list and potential updated lists to display
    new_non_zero_float_length = len(new_non_zero_float)

    # Creating a variable to create a print that will only print based on an earlier print function
    difference = 0

    print('Quantity of intially non-zero float entries:', non_zero_float_length, '\n')

    if non_zero_float_length !=0:

        # Checking if the quantity was marked as updated and checking if their was a difference
        # in the ammount of non-zero float entries. If so, it will display the new quantity.
        if (update == 'Y' or update == 'y') and non_zero_float != new_non_zero_float:
            print('Current Quantity of non-zero float entries:', new_non_zero_float_length, '\n')
            difference = 1
            print('First and last 10 intially non-zero float rows (with current values)')
    
        # If an update was requested but there wasn't a change in quantity then that will
        # be displayed to the user.
        elif update == 'Y' or update == 'y':
            print('Quanity of non-zero float entries has not changed \n')
            print('First and last 10 non-zero float rows (with current values)')
    
        else: 
            print('First and last 10 non-zero float rows and values:')
            
        # Now displaying the first 10 and last 10 
        for item in non_zero_float[:10]+ non_zero_float[-10:]:
            print(item, datfra[column][item])
    
        print('\n')
    
        # Checking if previous check found a new quantity of non-zero float entries, and then
        # Printing the first and last 10 entries of update
        if difference == 1:
            print('Current first and last 10 non-zero float rows and values:')
            for item in new_non_zero_float[:10]+ new_non_zero_float[-10:]:
                print(item, datfra[column][item])
    
            print('\n')

salary_wrong_rows = wrong_r('salary', df)

print_wrong_sample(salary_wrong_rows, salary_wrong_rows, 'salary', df, 'n')

salary_float_rows, salary_non_zero_float, salary_zero_float = float_r('salary', df)

print_float_sample(salary_float_rows, salary_float_rows, 'salary', df, 'n')

print_non_zero_float_sample(salary_non_zero_float, salary_non_zero_float, 'salary', df, 'n')



Quantity of intially incorrect entries: 32789 

First and last 10 incorrect rows and values:
200 Gina
1200 u
10000 nan
10001 nan
10002 nan
10003 nan
10004 nan
10005 nan
10006 nan
10007 nan
65526 61880.0
65527 51351.0
65528 53164.0
65529 58542.0
65530 62340.0
65531 111503.0
65532 58435.0
65533 92405.0
65534 88598.0
65535 55154.0


Quantity of intial float entries: 32767 

First and last 10 float rows and values:
32768 52534.0
32769 107036.0
32770 79556.0
32771 86706.0
32772 87428.0
32773 58813.0
32774 95583.0
32775 73689.0
32776 97965.0
32777 67031.0
65526 61880.0
65527 51351.0
65528 53164.0
65529 58542.0
65530 62340.0
65531 111503.0
65532 58435.0
65533 92405.0
65534 88598.0
65535 55154.0


Quantity of intially non-zero float entries: 0 



In [5]:
# All the entries that seem to be floats seem to just incorrectly contain an extra .0
# It seems like it would be a mistake to delete 32767 entries just for that reason.
# However I don't believe we were taught any protocols for imputing data nor do the instructions
# mention anything other than deleting incorrect data. I will create a parallel datafile.
df2 = df.copy()

# Creating a function that will take every innocious zero-float value and trim off the 
# '.0' while 
def zero_float_to_intstring(float_rows, column, datfra):
    for x in float_rows:
        datfra.loc[x, column] = str(datfra[column][x]).replace('.0','',1)



In [6]:
# Using function to trim of the .0's for salary field and 
# turning all the float values into useable int values
zero_float_to_intstring(salary_float_rows, 'salary', df2)

In [7]:
# Checking how fixing the float values affected our lists of incorrect entries
new_salary_wrong_rows = wrong_r('salary', df2)

print_wrong_sample(salary_wrong_rows, new_salary_wrong_rows, 'salary', df2, 'y')

new_salary_float_rows, new_salary_non_zero_float, new_salary_zero_float = float_r('salary', df2)

print_float_sample(salary_float_rows, new_salary_float_rows, 'salary', df2, 'y')

print_non_zero_float_sample(salary_non_zero_float, new_salary_non_zero_float, 'salary', df2, 'y')

Quantity of intially incorrect entries: 32789 

Current Quantity of incorrect entries: 22 

First and last 10 intially incorrect rows (with current values)
200 Gina
1200 u
10000 nan
10001 nan
10002 nan
10003 nan
10004 nan
10005 nan
10006 nan
10007 nan
65526 61880
65527 51351
65528 53164
65529 58542
65530 62340
65531 111503
65532 58435
65533 92405
65534 88598
65535 55154


Current first and last 10 incorrect rows and values:
200 Gina
1200 u
10000 nan
10001 nan
10002 nan
10003 nan
10004 nan
10005 nan
10006 nan
10007 nan
10010 nan
10011 nan
10012 nan
10013 nan
10014 nan
10015 nan
10016 nan
10017 nan
10018 nan
10019 nan


Quantity of intial float entries: 32767 

Current Quantity of float entries: 0 

First and last 10 intial float rows (with current values)
32768 52534
32769 107036
32770 79556
32771 86706
32772 87428
32773 58813
32774 95583
32775 73689
32776 97965
32777 67031
65526 61880
65527 51351
65528 53164
65529 58542
65530 62340
65531 111503
65532 58435
65533 92405
65534 88598
65535

In [8]:
df.shape

(100003, 16)

In [9]:
# Now applying the same system to the experience_total column to see if there are similar
# problems
field = 'experience_total'

total_wrong_rows = wrong_r(field, df)

print_wrong_sample(total_wrong_rows, total_wrong_rows, field, df, 'n')

total_float_rows, total_non_zero_float, total_zero_float = float_r(field, df)

print_float_sample(total_float_rows, total_float_rows, field, df, 'n')

print_non_zero_float_sample(total_non_zero_float, total_non_zero_float, field, df, 'n')

Quantity of intially incorrect entries: 32789 

First and last 10 incorrect rows and values:
91 t
200 Gina
20000 nan
20001 nan
20002 nan
20003 nan
20004 nan
20005 nan
20006 nan
20007 nan
65526 40.0
65527 14.0
65528 40.0
65529 12.0
65530 27.0
65531 33.0
65532 10.0
65533 39.0
65534 40.0
65535 35.0


Quantity of intial float entries: 32767 

First and last 10 float rows and values:
32768 4.0
32769 23.0
32770 40.0
32771 26.0
32772 4.0
32773 33.0
32774 31.0
32775 14.0
32776 35.0
32777 0.0
65526 40.0
65527 14.0
65528 40.0
65529 12.0
65530 27.0
65531 33.0
65532 10.0
65533 39.0
65534 40.0
65535 35.0


Quantity of intially non-zero float entries: 0 



In [10]:
# Using our function to remove .0s from experience_total
zero_float_to_intstring(total_float_rows, field, df2)

In [11]:
# Checking how fixing the float values affected our lists of incorrect entries
new_total_wrong_rows = wrong_r(field, df2)

print_wrong_sample(total_wrong_rows, new_total_wrong_rows, field, df2, 'y')

new_total_float_rows, new_total_non_zero_float, new_total_zero_float = float_r(field, df2)

print_float_sample(total_float_rows, new_total_float_rows, field, df2, 'y')

print_non_zero_float_sample(total_non_zero_float, new_total_non_zero_float, field, df2, 'y')

Quantity of intially incorrect entries: 32789 

Current Quantity of incorrect entries: 22 

First and last 10 intially incorrect rows (with current values)
91 t
200 Gina
20000 nan
20001 nan
20002 nan
20003 nan
20004 nan
20005 nan
20006 nan
20007 nan
65526 40
65527 14
65528 40
65529 12
65530 27
65531 33
65532 10
65533 39
65534 40
65535 35


Current first and last 10 incorrect rows and values:
91 t
200 Gina
20000 nan
20001 nan
20002 nan
20003 nan
20004 nan
20005 nan
20006 nan
20007 nan
20010 nan
20011 nan
20012 nan
20013 nan
20014 nan
20015 nan
20016 nan
20017 nan
20018 nan
20019 nan


Quantity of intial float entries: 32767 

Current Quantity of float entries: 0 

First and last 10 intial float rows (with current values)
32768 4
32769 23
32770 40
32771 26
32772 4
32773 33
32774 31
32775 14
32776 35
32777 0
65526 40
65527 14
65528 40
65529 12
65530 27
65531 33
65532 10
65533 39
65534 40
65535 35


Current first and last 10 float rows and values:


Quantity of intially non-zero float entr

In [12]:
# Now applying the same system to the experience_nj column to see if there are similar
# problems
field = 'experience_nj'

nj_wrong_rows = wrong_r(field, df)

print_wrong_sample(nj_wrong_rows, nj_wrong_rows, field, df, 'n')

nj_float_rows, nj_non_zero_float, nj_zero_float = float_r(field, df)

print_float_sample(nj_float_rows, nj_float_rows, field, df, 'n')

print_non_zero_float_sample(nj_non_zero_float, nj_non_zero_float, field, df, 'n')

Quantity of intially incorrect entries: 100001 

First and last 10 incorrect rows and values:
0 9.0
1 13.0
2 7.0
3 26.0
4 5.0
5 20.0
6 39.0
7 1.0
8 1.0
9 15.0
99993 5.0
99994 34.0
99995 8.0
99996 36.0
99997 12.0
99998 10.0
99999 15.0
100000 39.0
100001 17.0
100002 11.0


Quantity of intial float entries: 100000 

First and last 10 float rows and values:
0 9.0
1 13.0
2 7.0
3 26.0
4 5.0
5 20.0
6 39.0
7 1.0
8 1.0
9 15.0
99993 5.0
99994 34.0
99995 8.0
99996 36.0
99997 12.0
99998 10.0
99999 15.0
100000 39.0
100001 17.0
100002 11.0


Quantity of intially non-zero float entries: 0 



In [13]:
# Using our function to remove .0s from experience_nj
zero_float_to_intstring(nj_float_rows, field, df2)

In [14]:
# Checking how fixing the float values affected our lists of incorrect entries
new_nj_wrong_rows = wrong_r(field, df2)

print_wrong_sample(nj_wrong_rows, new_nj_wrong_rows, field, df2, 'y')

new_nj_float_rows, new_nj_non_zero_float, new_nj_zero_float = float_r(field, df2)

print_float_sample(nj_float_rows, new_nj_float_rows, field, df2, 'y')

print_non_zero_float_sample(nj_non_zero_float, new_nj_non_zero_float, field, df2, 'y')

Quantity of intially incorrect entries: 100001 

Current Quantity of incorrect entries: 1 

First and last 10 intially incorrect rows (with current values)
0 9
1 13
2 7
3 26
4 5
5 20
6 39
7 1
8 1
9 15
99993 5
99994 34
99995 8
99996 36
99997 12
99998 10
99999 15
100000 39
100001 17
100002 11


Current first and last 10 incorrect rows and values:
200 Gina
200 Gina


Quantity of intial float entries: 100000 

Current Quantity of float entries: 0 

First and last 10 intial float rows (with current values)
0 9
1 13
2 7
3 26
4 5
5 20
6 39
7 1
8 1
9 15
99993 5
99994 34
99995 8
99996 36
99997 12
99998 10
99999 15
100000 39
100001 17
100002 11


Current first and last 10 float rows and values:


Quantity of intially non-zero float entries: 0 



In [15]:
# Now applying the same system to the experience_district column to see if there are similar
# problems
field = 'experience_district'

district_wrong_rows = wrong_r(field, df)

print_wrong_sample(district_wrong_rows, district_wrong_rows, field, df, 'n')

district_float_rows, district_non_zero_float, district_zero_float = float_r(field, df)

print_float_sample(district_float_rows, district_float_rows, field, df, 'n')

print_non_zero_float_sample(district_non_zero_float, district_non_zero_float, field, df, 'n')

Quantity of intially incorrect entries: 65535 

First and last 10 incorrect rows and values:
0 9.0
1 13.0
2 7.0
3 26.0
4 5.0
5 1.0
6 39.0
7 1.0
8 22.0
9 15.0
65527 7.0
65528 37.0
65529 11.0
65530 18.0
65531 4.0
65532 10.0
65533 7.0
65534 11.0
65535 21.0
86035 j


Quantity of intial float entries: 65533 

First and last 10 float rows and values:
0 9.0
1 13.0
2 7.0
3 26.0
4 5.0
5 1.0
6 39.0
7 1.0
8 22.0
9 15.0
65526 3.0
65527 7.0
65528 37.0
65529 11.0
65530 18.0
65531 4.0
65532 10.0
65533 7.0
65534 11.0
65535 21.0


Quantity of intially non-zero float entries: 0 



In [16]:
# Using our function to remove .0s from experience_district
zero_float_to_intstring(district_float_rows, field, df2)

In [17]:
# Checking how fixing the float values affected our lists of incorrect entries
new_district_wrong_rows = wrong_r(field, df2)

print_wrong_sample(district_wrong_rows, new_district_wrong_rows, field, df2, 'y')

new_district_float_rows, new_district_non_zero_float, new_district_zero_float = float_r(field, df2)

print_float_sample(district_float_rows, new_district_float_rows, field, df2, 'y')

print_non_zero_float_sample(district_non_zero_float, new_district_non_zero_float, field, df2, 'y')

Quantity of intially incorrect entries: 65535 

Current Quantity of incorrect entries: 2 

First and last 10 intially incorrect rows (with current values)
0 9
1 13
2 7
3 26
4 5
5 1
6 39
7 1
8 22
9 15
65527 7
65528 37
65529 11
65530 18
65531 4
65532 10
65533 7
65534 11
65535 21
86035 j


Current first and last 10 incorrect rows and values:
200 Gina
86035 j
200 Gina
86035 j


Quantity of intial float entries: 65533 

Current Quantity of float entries: 0 

First and last 10 intial float rows (with current values)
0 9
1 13
2 7
3 26
4 5
5 1
6 39
7 1
8 22
9 15
65526 3
65527 7
65528 37
65529 11
65530 18
65531 4
65532 10
65533 7
65534 11
65535 21


Current first and last 10 float rows and values:


Quantity of intially non-zero float entries: 0 



In [18]:
# Since experience_state would only have two entries if this protocol is not followe,
# I am going to assume that this is the correct protocol and will set df2 back to df 
# for the rest of the assignment.
df = df2

In [19]:
def wrong_r_dec(column, datfra):
    wrong_rows = []

    # Creating a for loop that will move through each entry in a column/field and check if it
    # contains anything that can't be reassigned to a float, and if it does it will add that
    # column to a list.
    for i in range(len(datfra)):

        # Since several completely blank rows have already been removed, the loop experiences
        # a range error when it runs into those rows, so first it will check if i matches one 
        # of the values from the dataframe and if it doesn't then that is a skipped row and
        # the loop will iterate immediately to prevent the range error and move onto the 
        # the next populated row, while keeping the index of the for-loop synced with the 
        # index of the dataframe.
        if i in datfra.index:
            # Creating a variable that will store the value from a certain cell so it can be 
            # checked if it contains anything that makes it not a float
            value = datfra[column][i]
            value_str = str(value)
            
            # I will use a try statement here instead of an if statement as the .isdigit()
            # function won't be able to easily analyze whether the entry meets the specific 
            # parameters of being a float. Try statements let us see if a task fails, and if so
            # we can do otherwise without creating an error. If it fails to convert value_float
            # into a float from a string then we will save it as an incorrect entry.
            try:
                value_float = float(value_str)
            except:
                wrong_rows.append(i)
                # We will add a continue just to make sure the code iterates correctly
                continue

        # Iterating if i doesn't match index i.e. row missing
        else:
            continue

    return wrong_rows


In [20]:
# Finding bad rows for fte.
field = 'fte'

fte_wrong_rows = wrong_r_dec(field, df)

print_wrong_sample(fte_wrong_rows, fte_wrong_rows, field, df, 'n')

Quantity of intially incorrect entries: 1 

First and last 10 incorrect rows and values:
200 Gina
200 Gina




In [21]:
# Replacing all the incorrect fields of the int columns
df[['salary', 'experience_district', 'experience_nj', 'experience_total']] = \
df[['salary', 'experience_district', 'experience_nj', 'experience_total']].replace\
('[^0-9]',np.nan,regex=True)

In [22]:
# Checking if all fields went to blank
print('Salary:')
blank_salary_wrong_rows = wrong_r('salary', df)

print_wrong_sample(new_salary_wrong_rows, blank_salary_wrong_rows, 'salary', df, 'y')

print('Experience Total:')

field = 'experience_total'
blank_total_wrong_rows = wrong_r(field, df)

print_wrong_sample(new_total_wrong_rows, blank_total_wrong_rows, field, df, 'y')

print('Experience NJ:')

field = 'experience_nj'
blank_nj_wrong_rows = wrong_r(field, df)

print_wrong_sample(new_nj_wrong_rows, blank_nj_wrong_rows, field, df, 'y')

print('Experience District:')

field = 'experience_district'
blank_district_wrong_rows = wrong_r(field, df)

print_wrong_sample(new_district_wrong_rows, blank_district_wrong_rows, field, df, 'y')



Salary:
Quantity of intially incorrect entries: 22 

Quanity of incorrect entries has not changed 

First and last 10 incorrect rows (with current values)
200 nan
1200 nan
10000 nan
10001 nan
10002 nan
10003 nan
10004 nan
10005 nan
10006 nan
10007 nan
10010 nan
10011 nan
10012 nan
10013 nan
10014 nan
10015 nan
10016 nan
10017 nan
10018 nan
10019 nan


Experience Total:
Quantity of intially incorrect entries: 22 

Quanity of incorrect entries has not changed 

First and last 10 incorrect rows (with current values)
91 nan
200 nan
20000 nan
20001 nan
20002 nan
20003 nan
20004 nan
20005 nan
20006 nan
20007 nan
20010 nan
20011 nan
20012 nan
20013 nan
20014 nan
20015 nan
20016 nan
20017 nan
20018 nan
20019 nan


Experience NJ:
Quantity of intially incorrect entries: 1 

Quanity of incorrect entries has not changed 

First and last 10 incorrect rows (with current values)
200 nan
200 nan


Experience District:
Quantity of intially incorrect entries: 2 

Quanity of incorrect entries has not cha

In [23]:
df = df.dropna()

In [24]:
# Creating a function that just allows the quantities to of errors to be checked and compared
# If the rows were just removed then trying to access their values will cause an error
def print_wrong_quant (wrong_rows, new_wrong_rows, column, datfra, update):

    # Assigning the length of both the original list and potential updated lists to display
    wrong_rows_length = len(wrong_rows)
    new_wrong_rows_length = len(new_wrong_rows)

    # Creating a variable to create a print that will only print based on an earlier print function
    difference = 0

    print('Quantity of intially incorrect entries:', wrong_rows_length, '\n')

    if wrong_rows_length !=0:
    
        # Checking if the quantity was marked as updated and checking if their was a difference
        # in the ammount of incorrect entries. If so, it will display the new quantity.
        if (update == 'Y' or update == 'y') and wrong_rows != new_wrong_rows:
            print('Current Quantity of incorrect entries:', new_wrong_rows_length, '\n')
            difference = 1
    
        # If an update was requested but there wasn't a change in quantity then that will
        # be displayed to the user.
        elif update == 'Y' or update == 'y':
            print('Quanity of incorrect entries has not changed \n')
    
        print('\n')

In [25]:
print('fte:')

field = 'fte'

new_fte_wrong_rows = wrong_r_dec(field, df)

print_wrong_quant(fte_wrong_rows, new_fte_wrong_rows, field, df, 'y')

fte:
Quantity of intially incorrect entries: 1 

Current Quantity of incorrect entries: 0 





In [26]:
# All incorrect rows with decimal places have already been removed so an extra expression is
# not necessary

In [27]:
# Checking if all fields went to blank
print('Salary:')
clear_salary_wrong_rows = wrong_r('salary', df)

print_wrong_quant(new_salary_wrong_rows, clear_salary_wrong_rows, 'salary', df, 'y')

print('Experience Total:')

field = 'experience_total'
clear_total_wrong_rows = wrong_r(field, df)

print_wrong_quant(new_total_wrong_rows, clear_total_wrong_rows, field, df, 'y')

print('Experience NJ:')

field = 'experience_nj'
clear_nj_wrong_rows = wrong_r(field, df)

print_wrong_quant(new_nj_wrong_rows, clear_nj_wrong_rows, field, df, 'y')

print('Experience District:')

field = 'experience_district'
clear_district_wrong_rows = wrong_r(field, df)

print_wrong_quant(new_district_wrong_rows, clear_district_wrong_rows, field, df, 'y')

Salary:
Quantity of intially incorrect entries: 22 

Current Quantity of incorrect entries: 0 



Experience Total:
Quantity of intially incorrect entries: 22 

Current Quantity of incorrect entries: 0 



Experience NJ:
Quantity of intially incorrect entries: 1 

Current Quantity of incorrect entries: 0 



Experience District:
Quantity of intially incorrect entries: 2 

Current Quantity of incorrect entries: 0 





In [28]:
# Setting the data type for all int columns
df = df.copy()
columns_to_convert = ['salary', 'experience_district', 'experience_nj', 'experience_total'] 
for col in columns_to_convert:
    df[col] = pd.to_numeric(df[col], downcast ='integer')

In [29]:
# Setting the data type for the float column, fte.
df['fte'] = pd.to_numeric(df['fte'], downcast ='float')

In [30]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
Index: 99954 entries, 0 to 100004
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   99954 non-null  float64
 1   last_name            99954 non-null  object 
 2   first_name           99954 non-null  object 
 3   county               99954 non-null  object 
 4   district             99954 non-null  object 
 5   school               99954 non-null  object 
 6   primary_job          99954 non-null  object 
 7   fte                  99954 non-null  float32
 8   salary               99954 non-null  int32  
 9   certificate          99954 non-null  object 
 10  subcategory          99954 non-null  object 
 11  teaching_route       99954 non-null  object 
 12  highly_qualified     99954 non-null  object 
 13  experience_district  99954 non-null  int8   
 14  experience_nj        99954 non-null  int8   
 15  experience_total     99954 non-null  int

(99954, 16)

In [31]:
# Checking the id column, hopefully every column should have an error but also be part of the zero float list
field = 'id'

id_wrong_rows = wrong_r(field, df)

print_wrong_sample(id_wrong_rows, id_wrong_rows, field, df, 'n')

id_float_rows, id_non_zero_float, id_zero_float = float_r(field, df)

print_float_sample(id_float_rows, id_float_rows, field, df, 'n')

print_non_zero_float_sample(id_non_zero_float, id_non_zero_float, field, df, 'n')

Quantity of intially incorrect entries: 99903 

First and last 10 incorrect rows and values:
0 1.0
1 2.0
2 3.0
3 4.0
4 5.0
5 6.0
6 7.0
7 8.0
8 9.0
9 10.0
99944 99943.0
99945 99944.0
99946 99945.0
99947 99946.0
99948 99947.0
99949 99948.0
99950 99949.0
99951 99950.0
99952 99951.0
99953 99952.0


Quantity of intial float entries: 99903 

First and last 10 float rows and values:
0 1.0
1 2.0
2 3.0
3 4.0
4 5.0
5 6.0
6 7.0
7 8.0
8 9.0
9 10.0
99944 99943.0
99945 99944.0
99946 99945.0
99947 99946.0
99948 99947.0
99949 99948.0
99950 99949.0
99951 99950.0
99952 99951.0
99953 99952.0


Quantity of intially non-zero float entries: 0 



In [32]:
all_id = df['id'].tolist()
other_id = list(set(all_id)-set(id_wrong_rows))
print(other_id)

[99968.0, 99969.0, 99970.0, 99971.0, 99972.0, 99973.0, 99974.0, 99975.0, 99976.0, 99977.0, 99978.0, 99979.0, 99980.0, 99981.0, 99982.0, 99983.0, 99984.0, 99961.0, 99985.0, 86035.0, 99986.0, 99987.0, 99988.0, 99989.0, 99990.0, 99991.0, 1562.0, 99992.0, 99993.0, 99994.0, 99995.0, 99996.0, 99997.0, 99998.0, 99999.0, 100000.0, 100001.0, 100002.0, 100003.0, 1200.0, 200.0, 91.0, 99954.0, 99955.0, 99956.0, 46709.0, 99957.0, 99959.0, 99960.0, 99958.0, 99962.0, 99963.0, 99964.0, 99965.0, 99966.0, 99967.0]


In [33]:
# I am not sure why these weren't returned by my program as well. Some 
# of them seem to have been deleted, but none of them are in the incorrect format

In [34]:
df['id'] = pd.to_numeric(df['id'], downcast ='integer')

In [35]:
field = 'id'

id_wrong_rows = wrong_r(field, df)

print_wrong_sample(id_wrong_rows, id_wrong_rows, field, df, 'n')

id_float_rows, id_non_zero_float, id_zero_float = float_r(field, df)

print_float_sample(id_float_rows, id_float_rows, field, df, 'n')

print_non_zero_float_sample(id_non_zero_float, id_non_zero_float, field, df, 'n')

Quantity of intially incorrect entries: 0 

Quantity of intial float entries: 0 

Quantity of intially non-zero float entries: 0 



In [36]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
Index: 99954 entries, 0 to 100004
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   99954 non-null  int32  
 1   last_name            99954 non-null  object 
 2   first_name           99954 non-null  object 
 3   county               99954 non-null  object 
 4   district             99954 non-null  object 
 5   school               99954 non-null  object 
 6   primary_job          99954 non-null  object 
 7   fte                  99954 non-null  float32
 8   salary               99954 non-null  int32  
 9   certificate          99954 non-null  object 
 10  subcategory          99954 non-null  object 
 11  teaching_route       99954 non-null  object 
 12  highly_qualified     99954 non-null  object 
 13  experience_district  99954 non-null  int8   
 14  experience_nj        99954 non-null  int8   
 15  experience_total     99954 non-null  int

(99954, 16)

## Testing and Formatting String Columns

In [37]:
start_space = df.index[df['first_name'].str.startswith(' ')].tolist()
print(start_space[:5])
for item in start_space[:1]+ start_space[-1:]:
    print(item, df['first_name'][item])



[40000, 40001, 40002, 40003, 40004]
40000  Christopher 
40119  Mallory 


In [38]:
end_space = df.index[df['first_name'].str.endswith(' ')].tolist()
print(end_space[:5])
for item in end_space[:1]+ end_space[-1:]:
    print(item, df['first_name'][item])

[40000, 40001, 40002, 40003, 40004]
40000  Christopher 
40119  Mallory 


In [39]:
columns_to_strip = ['last_name', 'first_name', 'county', 'district', 'school', 'primary_job', 'certificate', \
'subcategory', 'teaching_route', 'highly_qualified'] 
for col in columns_to_strip:
    df[col]= df[col].str.strip()

In [40]:
for item in start_space[:1]+ start_space[-1:]:
    print(item, df['first_name'][item])

40000 Christopher
40119 Mallory


## Finding Unwanted Characters

In [41]:
specials = [',', "'", '/', '"' '&', '(', ')', '-', ';', ':', '$', '.', '?', '!','[', ']', '%', \
            '^', '#']

# Checking for special characters in stricter fields to get a sense of what they might look
# like to guide what might be acceptable


def check_simple_specs(column):
    contains_speci = []
    for speci in specials:
        matches = df.index[df[column].str.contains(speci, regex=False)].tolist()
        contains_speci.extend(matches)
    print(contains_speci[:50])
    for item in contains_speci[:1]+ contains_speci[-1:]:
        print(item, df[column][item])

In [42]:


check_simple_specs('first_name')

check_simple_specs('last_name')

check_simple_specs('teaching_route')

check_simple_specs('subcategory')

[]
[]
[60000, 60001, 60002, 60003, 60004, 60005, 60006, 60007, 60008, 60009, 60010, 60011, 60012, 60013, 60014, 60015, 60016, 60017, 60018, 60019]
60000 Traditional éü 0.5556085214578451
60019 Traditional éü 0.39726189913927534
[]


In [43]:
# No special characters detected. Assuming other special characters beyond those tested
# are undesired

In [44]:
# Testing on a column expected to contain more characters
check_simple_specs('primary_job')

[22, 40, 124, 144, 157, 174, 277, 378, 430, 432, 446, 503, 608, 612, 690, 756, 800, 1002, 1017, 1071, 1073, 1077, 1122, 1171, 1251, 1259, 1275, 1297, 1518, 1572, 1689, 1709, 1779, 1794, 1824, 1840, 1915, 1966, 2003, 2095, 2106, 2113, 2141, 2238, 2263, 2307, 2326, 2446, 2492, 2668]
22 Elementary Teacher In Secondary Setting Apa Only Subcategory Identifier S, H, Or V
99904 Financial Literacy (math, Social Studies, Business:  Finance, Economics, Law, Comprehensive Business, Comprehensive Family And Consumer Sciences And G


In [45]:
df['first_name'] = df['first_name'].str.replace(r'[^A-Za-z\s]', '', regex=True)
df['last_name'] = df['last_name'].str.replace(r'[^A-Za-z\s]', '', regex=True)
df['teaching_route'] = df['teaching_route'].str.replace(r'[^A-Za-z\s]', '', regex=True)
df['subcategory'] = df['subcategory'].str.replace(r'[^A-Za-z\s]', '', regex=True)

In [46]:
check_simple_specs('first_name')

check_simple_specs('last_name')

check_simple_specs('teaching_route')

check_simple_specs('subcategory')

[]
[]
[]
[]


In [47]:
# Now we will extend the criteria to include allow for some apecial characters
def check_extra_specs(column):
    contains_speci = []
    for speci in specials:
        matches = df.index[df[column].str.contains(r"[^0-9A-Za-z\s,\'\/\"&\(\)\-\;\:\$\.\?\!\\\@\[\]%\^#]", regex=True)].tolist()
        contains_speci.extend(matches)
    print(contains_speci[:100])
    for item in contains_speci[:5]+ contains_speci[-5:]:
        print(item, df[column][item])
    print('Total with issue:', len(contains_speci))

In [48]:
check_extra_specs('primary_job')

[3023, 3195, 3212, 3543, 3644, 4128, 4164, 4299, 4600, 4772, 4889, 4910, 5316, 5682, 5689, 5891, 6030, 6558, 6835, 6954, 7104, 8172, 8697, 9097, 9329, 9438, 9612, 9667, 9896, 10456, 10956, 13184, 13370, 13711, 13867, 14095, 14204, 14344, 15031, 15703, 15774, 15851, 16145, 16183, 16587, 16685, 16713, 16900, 16922, 17491, 17602, 18450, 18985, 19052, 19264, 19890, 19943, 19994, 20103, 20264, 20368, 20620, 21348, 21416, 21733, 21959, 22004, 22089, 22493, 22551, 22607, 22733, 23056, 23466, 23629, 24402, 24641, 25639, 25755, 25825, 26002, 26199, 26732, 26782, 26820, 26989, 27119, 28460, 29168, 29388, 29565, 29663, 29768, 29776, 30205, 30289, 30424, 30484, 30779, 30852]
3023 Family & Consumer Sciences Ã¢â‚¬â€œ Apparel, Textiles And Interiors
3195 Family & Consumer Sciences Ã¢â‚¬â€œ Apparel, Textiles And Interiors
3212 Family & Consumer Sciences Ã¢â‚¬â€œ Apparel, Textiles And Interiors
3543 Family & Consumer Sciences Ã¢â‚¬â€œ Apparel, Textiles And Interiors
3644 Family & Consumer Sciences Ã¢â‚

In [49]:
check_extra_specs('county')
check_extra_specs('district')
check_extra_specs('school')
check_extra_specs('certificate')
check_extra_specs('highly_qualified')

[]
Total with issue: 0
[]
Total with issue: 0
[1849, 17420, 20127, 22149, 23155, 29145, 32398, 34020, 34960, 38514, 46363, 47684, 58519, 60547, 66889, 68157, 82805, 89493, 1849, 17420, 20127, 22149, 23155, 29145, 32398, 34020, 34960, 38514, 46363, 47684, 58519, 60547, 66889, 68157, 82805, 89493, 1849, 17420, 20127, 22149, 23155, 29145, 32398, 34020, 34960, 38514, 46363, 47684, 58519, 60547, 66889, 68157, 82805, 89493, 1849, 17420, 20127, 22149, 23155, 29145, 32398, 34020, 34960, 38514, 46363, 47684, 58519, 60547, 66889, 68157, 82805, 89493, 1849, 17420, 20127, 22149, 23155, 29145, 32398, 34020, 34960, 38514, 46363, 47684, 58519, 60547, 66889, 68157, 82805, 89493, 1849, 17420, 20127, 22149, 23155, 29145, 32398, 34020, 34960, 38514]
1849 Salome UreÃƒÂ±a Elementary School
17420 Salome UreÃƒÂ±a Elementary School
20127 Salome UreÃƒÂ±a Elementary School
22149 Salome UreÃƒÂ±a Elementary School
23155 Salome UreÃƒÂ±a Elementary School
60547 Salome UreÃƒÂ±a Elementary School
66889 Salome UreÃƒÂ±

In [50]:
df['county'] = df['county'].str.replace(r'[^A-Za-z\s]', '', regex=True)
df['district'] = df['district'].str.replace(r'[^A-Za-z\s]', '', regex=True)
df['school'] = df['school'].str.replace(r'[^A-Za-z\s]', '', regex=True)
df['certificate'] = df['certificate'].str.replace(r'[^A-Za-z\s]', '', regex=True)
df['highly_qualified'] = df['highly_qualified'].str.replace(r'[^A-Za-z\s]', '', regex=True)
df['primary_job'] = df['primary_job'].str.replace(r'[^A-Za-z\s]', '', regex=True)

In [51]:
check_extra_specs('primary_job')
check_extra_specs('county')
check_extra_specs('district')
check_extra_specs('school')
check_extra_specs('certificate')
check_extra_specs('highly_qualified')

[]
Total with issue: 0
[]
Total with issue: 0
[]
Total with issue: 0
[]
Total with issue: 0
[]
Total with issue: 0
[]
Total with issue: 0


In [52]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
Index: 99954 entries, 0 to 100004
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   99954 non-null  int32  
 1   last_name            99954 non-null  object 
 2   first_name           99954 non-null  object 
 3   county               99954 non-null  object 
 4   district             99954 non-null  object 
 5   school               99954 non-null  object 
 6   primary_job          99954 non-null  object 
 7   fte                  99954 non-null  float32
 8   salary               99954 non-null  int32  
 9   certificate          99954 non-null  object 
 10  subcategory          99954 non-null  object 
 11  teaching_route       99954 non-null  object 
 12  highly_qualified     99954 non-null  object 
 13  experience_district  99954 non-null  int8   
 14  experience_nj        99954 non-null  int8   
 15  experience_total     99954 non-null  int

(99954, 16)

## Dropping Duplicates

In [53]:
df = df.drop_duplicates()


In [54]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
Index: 99954 entries, 0 to 100004
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   99954 non-null  int32  
 1   last_name            99954 non-null  object 
 2   first_name           99954 non-null  object 
 3   county               99954 non-null  object 
 4   district             99954 non-null  object 
 5   school               99954 non-null  object 
 6   primary_job          99954 non-null  object 
 7   fte                  99954 non-null  float32
 8   salary               99954 non-null  int32  
 9   certificate          99954 non-null  object 
 10  subcategory          99954 non-null  object 
 11  teaching_route       99954 non-null  object 
 12  highly_qualified     99954 non-null  object 
 13  experience_district  99954 non-null  int8   
 14  experience_nj        99954 non-null  int8   
 15  experience_total     99954 non-null  int

(99954, 16)

## Saving Datafile

In [55]:
df.to_csv("cleaned_data.csv", index=False)


## Connecting to SQL and Creating Database from Cleansed Dataframe

In [72]:
#your code here
mydb = sq.connect(host="localhost",user="root",passwd="042497AS", buffered=True)
mycursor = mydb.cursor()

In [73]:
#your code here
mycursor.execute('USE nj_state_teachers_salaries')

In [62]:
#your code here
mycursor.execute("CREATE TABLE IF NOT EXISTS nj_state_teachers_salaries.teachers_salaries_pset4 \
(id INT NOT NULL, last_name TEXT, first_name TEXT, county TEXT, district TEXT, \
school TEXT, primary_job TEXT, fte FLOAT, salary INT, certificate TEXT, \
subcategory TEXT, teaching_route TEXT, highly_qualified TEXT, experience_district INT, \
experience_nj INT, experience_total INT, PRIMARY KEY (id))")

In [63]:
mydb.commit()

In [74]:
mycursor.execute("SET SESSION wait_timeout=28800") 
mycursor.execute("SET SESSION interactive_timeout=28800")

In [75]:
mycursor.execute("""
    LOAD DATA INFILE '/usr/local/mysql/upload/cleaned_data.csv' 
    INTO TABLE nj_state_teachers_salaries.teachers_salaries_pset4
    FIELDS TERMINATED BY ','
    OPTIONALLY ENCLOSED BY '"'  
    ESCAPED BY '\\\\'
    LINES TERMINATED BY '\\n'
    IGNORE 1 ROWS;
""")


In [76]:
mydb.commit()

## Checking Proper SQL Table Population

In [66]:
cmd = "select count(*) from \
                 nj_state_teachers_salaries.teachers_salaries_pset4 "
mycursor.execute(cmd)
count = mycursor.fetchone()[0]

print(f"Number of rows in teachers_salaries table : {count}")

Number of rows in teachers_salaries table : 99954


In [67]:
cmd = """SELECT COUNT(*) \
                FROM INFORMATION_SCHEMA.COLUMNS \
                WHERE table_schema = 'nj_state_teachers_salaries' \
                AND table_name = 'teachers_salaries_pset4'"""
mycursor.execute(cmd)
count = mycursor.fetchone()[0]
print(f"Number of columns in teachers_salaries table : {count}")        

Number of columns in teachers_salaries table : 16


## Creating Dataframe from CSV

In [79]:
#your code here
dff = pd.read_csv ('sample.csv')
print(dff.head(5))
print(dff.tail(5))
dff.shape

      id  last_name first_name     county                  district  \
0   5907  Velasquez       Carl  Hunterdon  South Hunterdon Regional   
1  91649      Evans      James     Mercer              Hamilton Twp   
2  33656      Davis     Amanda     Mercer  Hopewell Valley Regional   
3  37479  Mcconnell     Robert     Mercer  Hopewell Valley Regional   
4  63816    Francis     Elijah   Monmouth               Neptune Twp   

                                school                    primary_job  fte  \
0               West Amwell Twp School  Elementary Kindergraten Grade  0.5   
1             Hamilton Northnottingham    Elementary School Teacher K  0.8   
2  Hopewell Valley Central High School     Health  Physical Education  0.8   
3             Timberlane Middle School       Resource Program Inclass  1.0   
4  Shark River Hills Elementary School                    Music Vocal  0.8   

   salary           certificate subcategory teaching_route  \
0   74437  Standard certificate  Special e

(777, 16)

In [1]:
pip install nbconvert

Note: you may need to restart the kernel to use updated packages.
